In [29]:
import pandas as pd
import glob
import pytz
from pathlib import Path
import time

def convert_to_local(ts_utc, country_code):
    """Convert a UTC‐localized timestamp to the local timezone for a given country code."""
    try:
        tz_name = pytz.country_timezones[country_code][0]
    except (KeyError, IndexError):
        tz_name = "UTC"
    return ts_utc.tz_convert(tz_name)

def main():
    start = time.time()

    # 1. Discover files via glob
    data_dir = Path("/Users/baptistemeynet/Downloads/Projects/spotify_dashboard/Spotify Extended Streaming History")
    json_files = glob.glob(str(data_dir / "*.json"))
    print(f"\nTotal JSON files loaded: {len(json_files)}")

    # 2. Load & concatenate
    dfs = []
    for fp in json_files:
        dfs.append(pd.read_json(fp))
    df = pd.concat(dfs, ignore_index=True)

    # Initial row count
    counts = {}
    counts['initial'] = len(df)

    # 3. Filter out non-music media (podcasts/audiobooks)
    drop_cols = [
        "episode_name", "episode_show_name", "spotify_episode_uri",
        "audiobook_title", "audiobook_uri",
        "audiobook_chapter_uri", "audiobook_chapter_title"
    ]
    before = len(df)
    mask_has_media = df[drop_cols].notnull().any(axis=1)
    df = df[~mask_has_media]
    counts['after_media_filter'] = len(df)

    # 4. Drop unneeded columns
    df.drop(columns=drop_cols, inplace=True)

    # 5. Deduplicate
    before = len(df)
    df.drop_duplicates(keep="first", inplace=True)
    counts['after_dedup'] = len(df)

    # 6. Filter out rows with missing/blank track or artist
    before = len(df)
    track_blank = df['master_metadata_track_name'].fillna("").eq("")
    artist_blank = df['master_metadata_album_artist_name'].fillna("").eq("")
    df = df[~(track_blank | artist_blank)]
    counts['after_track_artist_filter'] = len(df)

    # 7. Timezone conversion
    df['ts_utc'] = pd.to_datetime(df['ts'], utc=True, errors='coerce')
    invalid_ts = df['ts_utc'].isna().sum()
    if invalid_ts > 0:
        df = df[df['ts_utc'].notna()]

    # Add tz_name column to identify if timezone resolution is successful
    df['tz_name'] = df['conn_country'].map(lambda code: pytz.country_timezones.get(code, ["UTC"])[0])
    df['tz_is_utc'] = df['tz_name'].eq("UTC")

    # Log invalid or missing countries
    invalid_country_mask = ~df['conn_country'].isin(pytz.country_timezones.keys())
    print(f"Rows with invalid or missing country codes: {invalid_country_mask.sum()}")

    # Convert to local time
    df['ts_local'] = df.apply(
        lambda row: convert_to_local(row['ts_utc'], row.get('conn_country', '')), axis=1
    )
    counts['remaining_utc'] = df['tz_is_utc'].sum()

    # 8. Reset index
    df.reset_index(drop=True, inplace=True)

    # 9. Summary
    print("\nSummary of row counts:")
    print(f"  Initial:             {counts['initial']}")
    print(f"  After media filter:  {counts['after_media_filter']}  (−{counts['initial'] - counts['after_media_filter']})")
    print(f"  After dedup:         {counts['after_dedup']}  (−{counts['after_media_filter'] - counts['after_dedup']})")
    print(f"  After track/artist:  {counts['after_track_artist_filter']}  (−{counts['after_dedup'] - counts['after_track_artist_filter']})")
    print(f"  Remaining UTC rows:  {counts['remaining_utc']}")

    # 10. Export to CSV
    out_path = data_dir / "final_spotify_history.csv"
    df.drop(columns=['ts_utc', 'tz_name', 'tz_is_utc'], inplace=True)
    df.to_csv(out_path, index=False)
    print(f"\nCleaned data written to: {out_path}")
    print(f"Script completed in {round(time.time() - start, 2)} seconds.")

if __name__ == "__main__":
    main()


Total JSON files loaded: 11
Rows with invalid or missing country codes: 1

Summary of row counts:
  Initial:             148263
  After media filter:  147258  (−1005)
  After dedup:         147137  (−121)
  After track/artist:  147137  (−0)
  Remaining UTC rows:  1

Cleaned data written to: /Users/baptistemeynet/Downloads/Projects/spotify_dashboard/Spotify Extended Streaming History/final_spotify_history.csv
Script completed in 17.03 seconds.
